### 1. Environment Setup

Before running the application, you need to set up your environment variables. This includes credentials for Modal, Hugging Face, and AWS. The following cells will help you create a `.env` file with the necessary credentials.

**Important:** Replace the placeholder values (`xxx`) with your actual credentials.

In [ ]:
MODAL_TOKEN_ID = "xxx"
MODAL_TOKEN_SECRET = "xxx"
MODAL_PROFILE = "eikona-io"
HF_TOKEN = "xxx"

AWS_ACCESS_KEY_ID = "xxx"
AWS_SECRET_ACCESS_KEY = "xxx"
AWS_DEFAULT_REGION = "eu-north-1"
AWS_S3_BUCKET = "eikona-io-ml"

In [ ]:
# Create a .env file with these contents
with open(".env", "w") as f:
    f.write(f"""MODAL_TOKEN_ID={MODAL_TOKEN_ID}
MODAL_TOKEN_SECRET={MODAL_TOKEN_SECRET}
MODAL_PROFILE={MODAL_PROFILE}
HF_TOKEN={HF_TOKEN}
AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
AWS_DEFAULT_REGION={AWS_DEFAULT_REGION}
AWS_S3_BUCKET={AWS_S3_BUCKET}
""")

### 2. Install Dependencies

Next, install the required Python packages using the `requirements.txt` file.

In [ ]:
!pip install -r requirements.txt

### 3. Run the Inference Server

Now, open a new terminal and start the inference server using the Modal CLI. The command will deploy the `flux_serve.py` script to the Modal serverless platform.

```sh
# Command to start the server:
modal serve flux_serve.py

# Expected output:
✓ Initialized. View run at https://modal.com/apps/eikona-io/main/ap-hRzmyUVVgEHP6WwiqmuyCB
✓ Created objects.
├── 🔨 Created mount /home/ubuntu/Modal-ML-Inference-Service/flux_serve.py
├── 🔨 Created mount PythonPackage:schemas, PythonPackage:utils
├── 🔨 Created web function fastapi_app => https://eikona-io--flux-fastapi-app-dev.modal.run
└── 🔨 Created function FluxModel.*.
️️⚡️ Serving... hit Ctrl-C to stop!
├── Watching /home/ubuntu/Modal-ML-Inference-Service/schemas.
├── Watching /home/ubuntu/Modal-ML-Inference-Service/utils.
└── Watching /home/Modal-ML-Inference-Service.
⠼ Running app...
```

### 4. Prepare the Inference Request

Now that the server is running, you can prepare the request payload. The following cell defines the parameters for the image generation request, such as the prompt, number of images, and diversity settings.

In [7]:
PROMPT = "a photo of a corgi surfing a wave at sunset"
GPUS = 2
NOF_IMAGES = 4
SAMPLERS = ["euler", "ddim", "dpmpp_2m"] # Only euler supported
NOF_STEPS = [10, 20, 30]
CFG_SCALES = [3.5, 5.0, 7.5]
SEEDS = 'mix' # Feature not implemented yet
STREAM = False # Feature not implemented yet
TOP_K = None # Feature not implemented yet

### 5. Send the Request and Get the Results

Finally, send the request to the inference server and print the response. The response will contain the URLs of the generated images and other metadata.

In [10]:
import requests

url = "https://eikona-io--flux-fastapi-app-dev.modal.run/generate"

payload = {
    "prompt": PROMPT,
    "nof_images": NOF_IMAGES,
    "gpus": GPUS,
    "diversity": {
        "samplers": SAMPLERS,
        "nof_steps": NOF_STEPS,
        "cfg_scales": CFG_SCALES,
        "seeds": SEEDS
    },
    "stream": STREAM,
    "top_k": TOP_K
}

resp = requests.post(url, json=payload, timeout=180)
resp.raise_for_status()  # raises if non-2xx
resp.json()

{'prompt': 'a photo of a corgi surfing a wave at sunset',
 'count': 4,
 'images': [{'url': 'https://eikona-io-ml.s3.eu-north-1.amazonaws.com/images/flux-1756310122703.jpg',
   'sampling_params': {'sampler': 'EulerDiscreteScheduler',
    'steps': 30,
    'cfg_scale': 5.0,
    'seed': 422596259}},
  {'url': 'https://eikona-io-ml.s3.eu-north-1.amazonaws.com/images/flux-1756310126768.jpg',
   'sampling_params': {'sampler': 'EulerDiscreteScheduler',
    'steps': 10,
    'cfg_scale': 5.0,
    'seed': 771367970}},
  {'url': 'https://eikona-io-ml.s3.eu-north-1.amazonaws.com/images/flux-1756310122746.jpg',
   'sampling_params': {'sampler': 'EulerDiscreteScheduler',
    'steps': 30,
    'cfg_scale': 3.5,
    'seed': 1960236773}},
  {'url': 'https://eikona-io-ml.s3.eu-north-1.amazonaws.com/images/flux-1756310126713.jpg',
   'sampling_params': {'sampler': 'EulerDiscreteScheduler',
    'steps': 20,
    'cfg_scale': 3.5,
    'seed': 1225293127}}],
 'latency_ms': 8034}